In [1]:
import time
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
# Set up Selenium options and driver
options = Options()
options.add_argument("--disable-gpu")
# options.add_argument("--headless")  # Optional: Run in headless mode

service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)

# Navigate to the page
base_url = "https://www.nykaa.com/luxe/brands/laneige/c/8242?"
params = "intcmp=brand_menu,luxe,Laneige"
paginationNumber = 1

# Allow initial time for the page to load
time.sleep(5)

# Scroll incrementally until all products are loaded
scroll_pause_time = 2  # Wait time after each scroll

# Lists to store data
productLinks = []
products = []

while True:

    if paginationNumber == 1:
        url = f"{base_url}{params}"
    else:
        url = f"{base_url}page_no={paginationNumber}&sort=popularity&intcmp=brand_menu,luxe,Laneige&eq=desktop"
    
    # Load the Page
    driver.get(url)
    print(f"Scraping Page {paginationNumber}: {url}")
    time.sleep(3)
    
    # Scroll to the bottom of the page
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(scroll_pause_time)

    # Wait until the products are loaded
    try:
        WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, '.productWrapper.css-17nge1h'))
        )
    except TimeoutException:
        print(f"Scraped the Total number of pages: {paginationNumber - 1}")
        break

    # Parse the page content with BeautifulSoup
    content = driver.page_source
    soup = BeautifulSoup(content, 'html.parser')
    
    # Find all product items
    deygaOrganicProducts = soup.findAll('div', class_='productWrapper css-17nge1h')
    if not deygaOrganicProducts:
        print("No Products found on this page.")
        break

    # Extract product links
    for product in deygaOrganicProducts:
        # Extract the Stock Status
        stockTag = product.find('span', class_ = "css-lg5xc9")
        if stockTag:
            stock = stockTag.get_text(strip = True)
        else:
            stock = "In Stock"

        # Extract the Product Highlight
        highlightTag = product.find("li", class_ = "custom-tag css-1bse542")
        if highlightTag:
            highlight = highlightTag.get_text(strip = True)
        else:
            highlight = "None"

        # Extract product link
        linkTag = product.find('a', href=True)
        if linkTag:
            productLink = linkTag['href']
            # Check if the link is a relative URL and add the base URL if necessary
            if not productLink.startswith("http"):  # Relative URL
                productLink = "https://www.nykaa.com" + productLink
            productLinks.append({"link" : productLink, "stock" : stock, "highlight" : highlight})
                
    for productData in productLinks:
        
        link = productData["link"]
        stock = productData["stock"]
        highlight = productData["highlight"]
        
        driver.get(link)
        time.sleep(3)  # Allow time for the page to load

        # Parse individual product page
        productPageContent = driver.page_source
        productSoup = BeautifulSoup(productPageContent, 'html.parser')
    
        # Extract Product Name
        productTag = productSoup.find('h1', class_='css-1gc4x7i')
        if productTag:
            prodName = productTag.get_text(strip=True)
        else:
            prodName = "No Description Available"
    
        # Extract Ratings
        ratingTag = productSoup.find('div', class_="css-1m0y15j")
        if ratingTag:
            ratingText = ratingTag.find('div', class_="css-m6n3ou")
            if ratingText:
                ratings = ratingText.get_text(strip = True)
            else:
                ratings = "N/A"
        else:
            ratings = "N/A"

        # Extract the Ratings & Reviews Count
        ratingCountTag = productSoup.find('div', class_ = "css-1eip5u4")
        if ratingCountTag:
            ratingCount = ratingCountTag.get_text(strip = True)
        else:
            ratingCount = "None"

        # Extract the Offer Price and Original Price
        productDiv = productSoup.find('div', class_ = "css-1d0jf8e")

        # Extract the first two (span) tags
        if productDiv:
            spanTags = productDiv.find_all('span')
            # print(len(spanTags))

            originalPrice = "N/A"
            offerPrice = "N/A"

            if len(spanTags) == 2:
                originalPrice = spanTags[1].get_text(strip = True)
                offerPrice = "N/A"
            elif len(spanTags) == 4:
                originalPrice = spanTags[1].get_text(strip = True)
                offerPrice = spanTags[2].get_text(strip = True)

            # Print the Values
            # print(f"Original Price: {originalPrice}, Offer Price: {offerPrice}")

        else:
            print("Product div not found, defaulting to Unavailable for prices")

        # Extract the Discount
        discountTag = productSoup.find('span', class_ = "css-bhhehx")
        if discountTag:
            discount = discountTag.get_text(strip = True)
        else:
            discount = "No Discount"

        products.append({
            "Product Brand" : "Laneige",
            "Product Name" : prodName,
            "Product Rating" : ratings,
            "Product Rating & Review Count" : ratingCount,
            "Product Original Price" : originalPrice, 
            "Product Offer Price" : offerPrice, 
            "Product Discount" : discount,
            "Product Highlight" : highlight,
            "Product Stock Status" : stock
        })

    # Clear the product links list to avoid the duplication
    productLinks.clear()

    paginationNumber += 1 # Increment the Page Number
    time.sleep(5)

# Close the browser after scraping
driver.quit()

Scraping Page 1: https://www.nykaa.com/luxe/brands/laneige/c/8242?intcmp=brand_menu,luxe,Laneige
Scraping Page 2: https://www.nykaa.com/luxe/brands/laneige/c/8242?page_no=2&sort=popularity&intcmp=brand_menu,luxe,Laneige&eq=desktop
Scraping Page 3: https://www.nykaa.com/luxe/brands/laneige/c/8242?page_no=3&sort=popularity&intcmp=brand_menu,luxe,Laneige&eq=desktop
Scraping Page 4: https://www.nykaa.com/luxe/brands/laneige/c/8242?page_no=4&sort=popularity&intcmp=brand_menu,luxe,Laneige&eq=desktop
Scraped the Total number of pages: 3


In [5]:
# print(f"Products: \n {products}")

In [4]:
laneige = pd.DataFrame(products)
laneige

,Product Brand,Product Name,Product Rating,Product Rating & Review Count,Product Original Price,Product Offer Price,Product Discount,Product Highlight,Product Stock Status
0,Laneige,LANEIGE Love Sale Combo,4.4/5,6277ratings&743reviews,₹2650,₹1500,43% Off,None,In Stock
1,Laneige,LANEIGE Lip Sleeping Mask Berry(8g),4.6/5,25027ratings&2681reviews,₹600,N/A,No Discount,BESTSELLER,In Stock
2,Laneige,LANEIGE Mini Lip Glowy Balm Set(20g),4.6/5,5ratings&5reviews,₹1200,N/A,No Discount,None,In Stock
3,Laneige,LANEIGE Lip Glowy Balm Berry Fruits Rouges(10g),4.6/5,20605ratings&2205reviews,₹750,N/A,No Discount,BESTSELLER,In Stock
4,Laneige,Laneige Cream Skin Cerapeptide Toner & Moistur...,4.5/5,8600ratings&1348reviews,₹900,N/A,No Discount,BESTSELLER,In Stock
5,Laneige,LANEIGE Smores Kisses Set(18g),5/5,2ratings&2reviews,₹1250,N/A,No Discount,None,In Stock
6,Laneige,LANEIGE Hydro UV Defense Sunscreen Ex Broad Sp...,2/5,1ratings&1reviews,₹1850,N/A,No Discount,None,In Stock
7,Laneige,LANEIGE Strong Barrier Minis Set(80ml),4.6/5,14ratings&5reviews,₹2100,N/A,No Discount,None,In Stock
8,Laneige,LANEIGE Bouncy And Firm Sleeping Mask(25ml),4.5/5,754ratings&144reviews,₹1300,N/A,No Discount,BESTSELLER,In Stock
9,Laneige,LANEIGE Lip Sleeping Mask - Sweet Candy Ex(8g),4.6/5,672ratings&90reviews,₹600,N/A,No Discount,BESTSELLER,In Stock


In [10]:
# laneige.to_csv("C:/Users/Logiya Vidhyapathy/Documents/KGISL Data Science/Project Files and Documents/Capstone Project/Laneige Products_Nykaa.csv", index = False)